# Load and glance 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('/kaggle/input/ai-generated-essays-dataset/AI Generated Essays Dataset.csv')

In [2]:
print(f"▶︎ Dataset shape: {df.shape}")
print("Column types & missing counts:")
print(df.dtypes.to_frame("dtype").join(df.isna().sum().to_frame("n_missing")))
print(df.dtypes.to_frame("dtype").join(df.isna().sum().to_frame("n_missing")))

print("▶︎ Sample rows:")
display(df.sample(3, random_state=42))



▶︎ Dataset shape: (1460, 2)
Column types & missing counts:
            dtype  n_missing
text       object          0
generated   int64          0
            dtype  n_missing
text       object          0
generated   int64          0
▶︎ Sample rows:


,text,generated
892,The most basic and persistent form of transpor...,0
1105,Many people are not in favor of keeping the El...,0
413,"Dear State Senator, The Electoral College is a...",0
